In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import KBinsDiscretizer, RobustScaler, MinMaxScaler, OrdinalEncoder,LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from mlxtend.feature_extraction import PrincipalComponentAnalysis
from category_encoders import WOEEncoder, TargetEncoder
import matplotlib.pyplot as plt
import seaborn as sns

import gc; gc.enable()

In [2]:
dfo=pd.read_csv('exercise_05_train.csv')
pd.set_option('display.max_columns', 110) # this adjusts columns displayed

In [3]:
# How do i keep 2 copies of DF, one DF original and one DF for working (without loading them both into memory separately)
df=dfo.copy()

In [4]:
df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,y
0,0.963686,6.627185,-45.224008,9.477531,-3.216532,13.216874,9.754747,5.245851,-1.102918,-2.867482,-37.632285,-12.983484,-30.244259,0.293407,-4.808540,9.293345,1.646353,5.687031,-14.049164,-1.158595,-20.776226,-0.789881,-39.396519,1.235763,-5.403411,-0.012328,0.703963,-22.946374,0.114717,-1.108137,-7.513845,1.278470,-3.606321,1.615670,chrystler,thur,-1.774488,8.804852,2.136329,2.501032,-6.011242,$-865.28,-32.366234,-27.950057,-5.662614,0.02%,-4.820389,26.350661,-17.522879,14.420626,-68.076453,-47.173300,0.954900,-3.708620,4.131492,8.424414,42.155760,-0.772753,-40.332336,4.748333,-9.002899,7.083087,-12.264477,8.626495,0.065363,-0.124457,21.750625,-5.282121,sept.,6.153703,-6.272020,17.255821,5.782086,15.141420,33.884019,-2.389519,-1.766949,-17.897092,3.736479,40.082477,16.640042,1.680384,0.373888,-13.562891,13.158882,2.289092,-3.881315,-15.959124,2.940515,-2.555928,-37.601994,0.988829,0.313772,asia,1.380664,-16.388994,5.326730,4.187294,0.045549,-3.646841,0
1,-1.770062,-23.610459,-0.964003,-31.981497,-10.294599,-10.240251,-1.518888,-1.675208,0.498134,-0.614390,47.652135,6.567264,-30.410216,-1.301751,-6.042590,1.817864,4.379207,5.676816,-37.757544,2.164646,59.052864,-1.480575,-35.736992,-8.580044,4.206910,0.732038,-1.109053,15.986872,-0.534066,5.795439,-2.556791,2.990327,-1.603091,0.633938,volkswagon,thur,0.040507,6.645667,3.205207,3.567783,5.405315,$325.27,-58.117719,-49.488582,-50.291610,-0.01%,1.003284,10.761073,-16.411092,2.474421,-24.002610,7.760549,-0.167079,-5.148356,8.422932,2.882336,47.178502,-0.166740,-36.199657,-0.231633,-2.334688,-4.212509,1.039474,-7.251746,0.661180,-7.073426,-36.774709,-11.458640,July,10.030685,0.918319,-38.648511,-24.077239,16.991279,19.847963,1.106517,6.949901,15.688587,4.610910,-15.223066,4.880278,6.029540,-0.506606,-16.914889,-8.219612,1.639064,7.030400,-11.601869,-4.907436,-6.075066,-27.168761,-2.162863,1.809807,asia,2.500590,4.338834,-1.583225,-1.172417,0.011216,0.097180,0
2,9.962401,-8.349849,23.248891,-24.196879,8.937480,10.965000,-7.490596,-3.025094,0.595807,0.382732,5.629537,3.769767,17.623199,-0.396468,-10.022546,1.158419,12.353464,0.398796,-0.201112,-5.803935,7.104037,2.393360,-30.622601,-12.318385,-6.341881,-0.055468,-1.176393,-13.288725,-0.383693,0.122918,-4.791641,-10.250248,5.837668,-3.756878,bmw,thurday,0.448073,-2.639292,-1.940933,-11.196415,-7.971329,$743.91,-47.325525,17.089850,-20.998601,0.0%,1.270592,20.947667,-1.198462,-1.228983,-16.860396,62.630162,-0.520845,-0.484470,-16.271894,-11.448579,-5.907019,0.627614,-13.539267,2.706494,17.375989,-2.434721,-1.433001,7.734776,0.489224,0.535575,2.491412,-9.652969,July,16.090371,0.180281,10.326921,-12.845434,19.811216,-44.341947,-0.513749,-1.495533,3.605778,0.480531,39.328247,-19.626495,9.321199,1.036403,19.435797,-7.605212,-3.937989,3.702377,-2.246263,1.441523,-5.642358,-90.377656,1.779660,9.528113,asia,1.396475,7.839188,10.402396,1.288991,0.008209,-4.132316,0
3,-5.780709,-25.261584,1.383115,-11.786929,7.993078,-11.245752,-2.607351,-3.513896,-0.614235,-1.453979,-2.636676,-3.595789,-19.648688,0.393391,-3.470142,0.102685,-0.882429,-1.598535,12.869959,5.374145,30.723272,4.541173,-59.270400,-0.101281,4.451334,0.650121,-1.154275,-19.434740,1.471404,9.678197,4.149206,-2.814893,-10.966831,6.516250,nissan,thurday,-0.625046,1.918934,1.046202,-6.727565,8.522306,$538.48,-36.489186,3.120606,-31.523483,0.01%,12.551744,0.908244,-8.868663,-37.101607,40.757401,-22.994817,0.532649,4.227293,6.650745,-0.398449,-9.192991,-0.935667,-61.343401,-14.767857,9.686716,5.354954,0.002768,9.552721,-4.347074,1.513008,14.245827,1.555428,July,10.215072,5.658302,-39.728801,-22.905705,34

In [5]:
# to select sub-dataframe with only numerical or object features
df.select_dtypes(include=['float64', 'int64'])
df.select_dtypes(include=['object'])

,x34,x35,x41,x45,x68,x93
0,chrystler,thur,$-865.28,0.02%,sept.,asia
1,volkswagon,thur,$325.27,-0.01%,July,asia
2,bmw,thurday,$743.91,0.0%,July,asia
3,nissan,thurday,$538.48,0.01%,July,asia
4,volkswagon,wed,$-433.65,0.0%,Jun,asia
...,...,...,...,...,...,...
39995,volkswagon,thurday,$60.77,-0.0%,Jun,asia
39996,tesla,thur,$-982.23,0.01%,Aug,asia
39997,volkswagon,thurday,$1282.79,-0.0%,Aug,asia
39998,volkswagon,thurday,$-282.31,0.01%,Aug,euorpe


In [11]:
print(df.shape)
df.dtypes.value_counts()

(40000, 101)


float64    94
object      6
int64       1
dtype: int64

In [7]:
df.info()

(40000, 101)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 101 entries, x0 to y
dtypes: float64(94), int64(1), object(6)
memory usage: 30.8+ MB


In [12]:
# is our class / target binary or multinomial
print(df.y.unique(),'\n')

# target value counts percentage to check for imbalance
df.y.value_counts(normalize=True) * 100
# df.y.value_counts()

[0 1] 



0    79.6325
1    20.3675
Name: y, dtype: float64

## How wide-ranging are the categorical columns

In [13]:
print('\n Feature x34 has:',df['x34'].nunique(),'unique values')
print('\n Feature x35 has:',df['x35'].nunique(),'unique values')
print('\n Feature x45 has:',df['x45'].nunique(),'unique values')
print('\n Feature x68 has:',df['x68'].nunique(),'unique values')
print('\n Feature x93 has:',df['x93'].nunique(),'unique values')
print('\n Feature x41 has:',df['x41'].nunique(),'unique values')
print('\n Feature TARGET Y has:',df['y'].nunique(),'unique values')


 Feature x34 has: 10 unique values

 Feature x35 has: 8 unique values

 Feature x45 has: 10 unique values

 Feature x68 has: 12 unique values

 Feature x93 has: 3 unique values

 Feature x41 has: 37817 unique values

 Feature TARGET Y has: 2 unique values


In [30]:
print('\nx34 \n',df['x34'].value_counts(normalize=True)*100)
print('\nx35 \n',df['x35'].value_counts(normalize=True)*100)
print('\nx45 \n',df['x45'].value_counts(normalize=True)*100)
print('\nx68 \n',df['x68'].value_counts(normalize=True)*100)
print('\nx93 \n',df['x93'].value_counts(normalize=True)*100)
print('\nx41 \n',df['x41'].value_counts(normalize=True)*100)
print('\nTARGET \n',df['y'].value_counts())


x34 
 volkswagon    31.561312
toyota        27.425485
bmw           18.158632
honda         12.937588
tesla          5.618624
chrystler      2.978096
nissan         0.815163
ford           0.400080
mercades       0.077516
chevrolet      0.027506
Name: x34, dtype: float64

x35 
 wednesday    51.907977
thursday     44.391098
tuesday       2.210553
friday        1.357839
monday        0.132533
Name: x35, dtype: float64

x45 
  0.00    38.881804
 0.01    24.029205
-0.01    23.871678
 0.02     5.933538
-0.02     5.908534
-0.03     0.697622
 0.03     0.607606
-0.04     0.042507
 0.04     0.027505
Name: x45, dtype: float64

x68 
 july       27.791253
jun        23.297742
aug        20.429597
may        11.862669
sept        8.761971
apr         4.073417
oct         2.212998
mar         1.017729
nov         0.362582
feb         0.120027
dev         0.040009
january     0.030007
Name: x68, dtype: float64

x93 
 asia       88.475483
america     7.918886
euorpe      3.605631
Name: x93, dtype: fl

In [15]:
## Set all words to lowercase for all columns with text. Is there a way to do all text columns at once?
df.x34 = df.x34.str.lower() ## this worked because we converted it into string first, then lowered. this is because for base python to LOWER, it needs to communicate with pandas, and KNOW that what it's converting is a string??? something anup explained.

In [16]:
# clean dirty days of the week
# TRY: IS THERE A BETTER WAY TO DO THIS than line for line?
# df.x35 = df.x35.replace('thur','thursday')
df.x35 = df.x35.replace(['thur','thurday'],'thursday')
df.x35 = df.x35.replace('fri','friday')
df.x35 = df.x35.replace('wed','wednesday')

In [17]:
df.x68 = df.x68.str.lower()
df.x68 = df.x68.str.rstrip('.') # in this case, ONLY sept has a dot at the end like 'sept.' but what if it was really messy with , or. or - and various characters. How do you delete ANYTHING/EVERYTHING after the end of the word?

In [18]:
# it's not mandatory to change the dev to dec, or january to jan, as these are obvious values and we 'know' what they mean

In [19]:
df.x41 = df.x41.str.strip('$')
# %timeit df.x41 = df.x41.str.replace('$','')

In [20]:
df.x41 = df.x41.astype(float)
df.x41.dtype

dtype('float64')

In [21]:
df.x45 = df.x45.str.rstrip('%')
df.x45 = df.x45.astype(float)

In [22]:
df['x41'].max()
#  IT WORKS NOW (AFTER CONVERTING IT TO FLOAT FROM OBJECT)

4327.53

In [23]:
df['x41'].min()

-4496.46

In [24]:
df.x41.sample(10)

5080      641.10
28535     539.50
3694    -1125.97
13901    -111.97
38570   -1396.95
9476       51.86
38756   -1219.78
20473    -243.22
34606    -720.09
13152    -868.84
Name: x41, dtype: float64

In [ ]:
# %timeit df['maximum'] = df.apply(lambda x: max(x['x1'], x['x2']), axis = 1)

In [ ]:
# %timeit df.max = df[['x1','x2']].max(axis =1)

In [ ]:
# x41: the $ column ranges from -4496.46 to 4327.53
# what does this mean? 
# is this balance due? 
# is this insurance loss?

In [25]:
objcols= [c for c in df.columns.tolist() if c in df.select_dtypes(include=['object'])]
objcols

['x34', 'x35', 'x68', 'x93']

In [26]:
df[objcols].describe()

,x34,x35,x68,x93
count,39992,39990,39991,39993
unique,10,5,12,3
top,volkswagon,wednesday,july,asia
freq,12622,20758,11114,35384


In [ ]:
df[objcols].isna().sum()

In [ ]:
df[['x34', 'x35','x68','x93']] = df[['x34', 'x35','x68','x93']].fillna(method='ffill')

In [ ]:
df[objcols].isna().sum()

In [ ]:
df.x34.isna().sum()

In [ ]:
numcols = [c for c in df.columns.tolist() if c not in objcols]
df[numcols]

In [ ]:
df[numcols] =df[numcols].fillna(method='ffill') #SettingWithCopyWarning: can be ignored.

In [ ]:
df[numcols].isna().sum()

In [ ]:
df.isna().sum().sum()

In [ ]:
# show value counts for each column in object type columns in desc order
frames = []
for c in objcols:
    if c != 'x41':
        frame = df[c].value_counts().to_frame().sort_values(by=c, ascending=False)
        frames.append(frame)
    
frames[0].T
frames[1].T

In [ ]:
frames = []
for c in objcols:
    if c != 'x41':
        frame = df[c].value_counts().to_frame().sort_values(by=c, ascending=False)
        frames.append(frame)
    
print(frames[0].T,'\n')
print(frames[1].T,'\n')
print(frames[2].T,'\n')
print(frames[3].T,'\n')
print(frames[4].T,'\n')

## Q - how do i print frames 0-4 in a better way?
## Q - it's 'nicer' to look at WITHOUT the print statement, how?

In [ ]:
for frame in frames:
    print(frame.head().T)

In [ ]:
for frame in frames:
    print(frame.tail().T)

In [ ]:
target = 'y'
feat_cols = [c for c in df.columns.tolist() if c not in [target]]
cnts_cols = [c for c in feat_cols if df[c].dtype == np.float]

In [ ]:
print(len(feat_cols))
print(len(cnts_cols))

In [ ]:
featcols = df.select_dtypes(include=['float64','int64']).copy()
featcols.head()

In [ ]:
featcols.isna().sum()

In [ ]:
catcols = df.select_dtypes(include=['object']).copy()
catcols.head()

In [ ]:
catcols.isna().values.sum()

In [ ]:
catcols.isna().sum()

In [ ]:
gc.collect()

In [ ]:
X, y = df[cnts_cols].values, df[target].values

imputer = SimpleImputer()
binner = KBinsDiscretizer(n_bins=10, encode='onehot-dense') # 80% with 2 bins.
# poly = PolynomialFeatures(interaction_only=True, include_bias=False)
lr = LogisticRegression(class_weight='balanced', n_jobs=-1, random_state=42)
pipe = make_pipeline(imputer, binner, lr) # poly

scores = cross_val_score(pipe, X, y, cv=5, scoring='roc_auc', n_jobs=-1)
scores.mean(), scores.std()

In [ ]:
X.shape

In [ ]:
# doing a corr plot is not a good idea in this case, because hard to see anything since we're dealing with ~100 columns
colormap = plt.cm.magma
plt.figure(figsize=(16,12))
plt.title('correlation of all df features', y=1.05, size=15)
sns.heatmap(df.corr(),linewidths=0.1,vmax=1.0, square=True, 
            cmap=colormap, linecolor='white', annot=True)

In [ ]:
pd.set_option('display.max_rows', None)
df.corr()[target].abs().sort_values(ascending=False)

## Using ALL Features

In [ ]:
X, y = df[feat_cols], df[target]

In [ ]:
from sklearn.compose import ColumnTransformer
# from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV


In [ ]:
# dont need scaling because we already have binning
numeric_transformer = Pipeline(steps=[
    ('binner',KBinsDiscretizer(n_bins=10, encode='onehot-dense'))])


In [ ]:
categorical_transformer = Pipeline(steps=[
    ('tencoder',TargetEncoder())])


In [ ]:
cat_cols = list(catcols.columns)

In [ ]:
cat_cols

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, cnts_cols),
        ('cat', categorical_transformer, cat_cols)])

In [ ]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(class_weight='balanced', n_jobs=-1, random_state=42))])

In [ ]:
scores = cross_val_score(clf, X, y.values, cv=5, scoring='roc_auc', n_jobs=-1)
scores.mean(), scores.std()

In [ ]:
scores

In [ ]:
# added object columns to baseline, but getting NaNs as result.
# added fill_value=None, still Nans
# tried encode=Ordinal, nothing
# added scaler just cuz, nothing

# X, y = df[feat_cols].values, df[target].values

# encoder = Label()
# scaler = RobustScaler()
# scaler2 = MinMaxScaler()
# pca = PrincipalComponentAnalysis(n_components=None,solver='svd')
# oencoder = OrdinalEncoder()
# wencoder = WOEEncoder()
# tencoder = TargetEncoder()
# lencoder = LabelEncoder()
# imputer = SimpleImputer(fill_value=None)
# binner = KBinsDiscretizer(n_bins=2, encode='onehot-dense')

lr = LogisticRegression(class_weight='balanced', n_jobs=-1, random_state=42) # penalty='l1', C=0.1, solver='liblinear'
pipe = make_pipeline(preprocessor,lr) # poly

scores = cross_val_score(pipe, X, y, cv=5, scoring='roc_auc', n_jobs=-1)
scores.mean(), scores.std()

In [ ]:
test=oencoder.fit_transform(X)

In [ ]:
test=pipe.fit_transform(X,y)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# test = imputer.fit_transform(X) # uses MEAN, so does not work with non-numeric data. cant convert string to float 'chrysler'

In [ ]:
test = oencoder.fit_transform(X) # input contains NAN (even though categorical columns have been fillna'd using ffill)

In [ ]:
pca  = PCA()

In [ ]:
test = pca.fit_transform(X) # could not convert string to float CHRYSLER

## PANDAS Practice

In [ ]:
# for col x34, give me the value in index row location 4
df.x34.iloc[4]
df.x34.iloc[[4]] # this syntax works but doesnt return DF, because we only want one field. but this result is different from above result. (includes index #)
# for col x34, give me the values in index row locations 2-4
df.x34.iloc[2:4]
# df.x34.iloc[[2:4]] - this does not work

In [ ]:
# for the whole dataset, pull index row 2
df.iloc[2].head()
# for the whole dataset, pull index row 2, display as dataframe
df.iloc[[2]]
# for the whole dataset, give me row location 2 and 4
df.iloc[[2,4]]
df.loc[[2,4]]

## INVESTIGATE

In [ ]:
df['x41'].max()
# df['x41'].min()
# this returns error: '>=' not supported between instances of 'str' and 'float'
# probably because the column is object type but since we removed the $ and now have numbers, it should be float?? let's try converting it to float below

In [ ]:
# feature x41 look like dollar amounts (continuous), investigating this column
# df.max(axis=0)[x41]
# df.max(axis=0)['x41']
# df['x41'].max()
# df['x41'].min()
df.x41.dtype
# min max isnt working because the values in column are not numeric, will drop the '$' sign then maybe convert column to float?

In [ ]:
## Set all words to lowercase for all columns with text
df.x34=df.x34.apply(lambda x: x.lower())
## DID NOT WORK, not sure how to convert it to string first.

In [ ]:
# what fucking TYPE is x34?
df.x34.dtype

In [ ]:
df.dtypes
## INVESTIGATE: this shows only floats, how do you check for df dtypes that != float64.

In [ ]:
df.x41.dtype
# yes this shows me that x41 is dtype('O') but what if i want to see it specifically mentioned what O means?

In [ ]:
df.select_dtypes(include!=['float64'])
df.dtypes(include != ['float64'])
# df.select_dtypes(include=['object'])

In [ ]:
g= df.columns.to_series().groupby(df.dtypes).groups
{k.name: v for k, v in g.items()}

In [ ]:
# select * columns of a certain type, e.g. object, float, int, bool
df.select_dtypes(include=['object'])

## WHY ARE THESE NOT WORKING???

In [ ]:
%timeit df.x41.str.apply(lambda x: x.replace('$',''))

In [ ]:
%timeit df.x41.apply(lambda x: x.strip('$'))

In [ ]:
%timeit [x.strip('$') for x in df.x41]

## FAIL #1

In [ ]:
# ran for 15+ minutes, showed 'busy'. might be a better idea to run on a smaller dataset like test instead of train. check if it works. aim is check value counts for all object columns but this was checking ALL.
df.apply(pd.Series.value_counts)

In [ ]:
# dfvc = pd.DataFrame(columns=df.columns)
df[objcols].apply(pd.Series.value_counts)
df[['x34', 'x35', 'x41', 'x45', 'x68', 'x93']].apply(pd.Series.value_counts)
# both lines display same thing. this was not displaying anything intelligible so bryan created functions (blocks under)